# Component Separation on CMB S4 and QUBIC+

In [ ]:
import qubicplus
import pysm3
import pysm3.units as u
from pysm3 import utils
import numpy as np
from qubic import camb_interface as qc
import healpy as hp
import matplotlib.pyplot as plt
import os
import random as rd
import string
import qubic
from importlib import reload
import pickle
import s4bi
from scipy import constants
from fgbuster import CMB, Dust, Synchrotron, AnalyticComponent

center = qubic.equ2gal(0, -57)
# If there is not this command, the kernel shut down every time..
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

nside=256
def get_coverage(fsky, nside, center_radec=[0., -57.]):
    center = qubic.equ2gal(center_radec[0], center_radec[1])
    uvcenter = np.array(hp.ang2vec(center[0], center[1], lonlat=True))
    uvpix = np.array(hp.pix2vec(nside, np.arange(12*nside**2)))
    ang = np.arccos(np.dot(uvcenter, uvpix))
    indices = np.argsort(ang)
    okpix = ang < -1
    okpix[indices[0:int(fsky * 12*nside**2)]] = True
    mask = np.zeros(12*nside**2)
    mask[okpix] = 1
    return mask

covmap = get_coverage(0.03, nside)


thr = 0.1
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0

#from fgbuster import (separation_recipes, xForecast, CMB, Dust, Dust_2b, Synchrotron, FreeFree, PowerLaw,  # sky-fitting model
                      #basic_comp_sep)
import fgbuster as fgb
from fgbuster import basic_comp_sep, get_instrument


def separate(comp, instr, maps_to_separate, tol=1e-5, print_option=False):
    solver_options = {}
    solver_options['disp'] = False
    fg_args = comp, instr, maps_to_separate
    fg_kwargs = {'method': 'Nelder-Mead', 'tol': tol, 'options': solver_options}
    try:
        res = fgb.basic_comp_sep(*fg_args, **fg_kwargs)
    except KeyError:
        fg_kwargs['options']['disp'] = False
        res = fgb.basic_comp_sep(*fg_args, **fg_kwargs)
    if print_option:
        print()
        print("message:", res.message)
        print("success:", res.success)
        print("result:", res.x)
        
    return res


def reconvolve(maps, fwhms, ref_fwhm, verbose=False):
    if verbose:
        print('Reconvolution to common FWHM')
    sig_conv = np.sqrt(ref_fwhm**2 - fwhms**2)
    maps_out = np.zeros_like(maps)
    for i in range(len(fwhms)):
        if sig_conv[i] == 0:
            if verbose:
                print('Map {0:} fwhmin={1:6.3f} fwhmout={2:6.3f} => We do not reconvolve'.format(i,
                                                                                             fwhms[i], ref_fwhm))
            maps_out[i,:] = maps[i,:]
        else:
            if verbose:
                print('Map {0:} fwhmin={1:6.3f} fwhmout={2:6.3f} => We reconvolve with {3:6.3f}'.format(i,
                                                                                                    fwhms[i],
                                                                                                    ref_fwhm,
                                                                                                    sig_conv[i]))
            maps_out[i,:] = hp.smoothing(maps[i,:], fwhm=np.deg2rad(sig_conv[i]), pol=True, verbose=False)
    return maps_out



# Loading of dictionary from BI-CMBS4.ipynb 

### CMB-S4 config

freqs = np.array([20., 30., 40., 85., 95., 145., 155., 220., 270.])
bandwidth = np.array([5., 9., 12., 20.4, 22.8, 31.9, 34.1, 48.4, 59.4])
dnu_nu = bandwidth/freqs
beam_fwhm = np.array([11., 72.8, 72.8, 25.5, 25.5, 22.7, 22.7, 13., 13.])
mukarcmin_TT = np.array([16.5, 9.36, 11.85, 2.02, 1.78, 3.89, 4.16, 10.15, 17.4])
mukarcmin_EE = np.array([10.87, 6.2, 7.85, 1.34, 1.18, 1.8, 1.93, 4.71, 8.08])
mukarcmin_BB = np.array([10.23, 5.85, 7.4, 1.27, 1.12, 1.76, 1.89, 4.6, 7.89])
ell_min = np.array([30, 30, 30, 30, 30, 30, 30, 30, 30])
nside = np.array([512, 512, 512, 512, 512, 512, 512, 512, 512])
edges_min = freqs * (1. - dnu_nu/2)
edges_max = freqs * (1. + dnu_nu/2)
edges = [[edges_min[i], edges_max[i]] for i in range(len(freqs))]
s4_config = {
    'nbands': len(freqs),
    'frequency': freqs,
    'depth_p': 0.5*(mukarcmin_EE + mukarcmin_BB),
    'depth_i': mukarcmin_TT,
    'depth_e': mukarcmin_EE,
    'depth_b': mukarcmin_BB,
    'fwhm': beam_fwhm,
    'bandwidth': bandwidth,
    'dnu_nu': dnu_nu,
    'ell_min': ell_min,
    'nside': nside,
    'fsky': 0.03,
    'ntubes': 12,
    'nyears': 7.,
    'edges': edges,
    'effective_fraction': np.zeros(len(freqs))+1.
            }

def fct_subopt(nus):
    subnus = [150., 220]
    subval = [1.4, 1.2]
    fct_subopt = np.poly1d(np.polyfit(subnus, subval, 1))
    return fct_subopt(nus)

subnus = [150., 220]
subval = [1.4, 1.2]

def get_coverage(fsky, nside, center_radec=[0., -57.]):
    center = qubic.equ2gal(center_radec[0], center_radec[1])
    uvcenter = np.array(hp.ang2vec(center[0], center[1], lonlat=True))
    uvpix = np.array(hp.pix2vec(nside, np.arange(12*nside**2)))
    ang = np.arccos(np.dot(uvcenter, uvpix))
    indices = np.argsort(ang)
    okpix = ang < -1
    okpix[indices[0:int(fsky * 12*nside**2)]] = True
    mask = np.zeros(12*nside**2)
    mask[okpix] = 1
    return mask

def qubicify(config, qp_nsub, qp_effective_fraction):
    nbands = np.sum(qp_nsubs)
    qp_config = config.copy()
    for k in qp_config.keys():
        qp_config[k]=[]
    qp_config['nbands'] = nbands
    qp_config['fsky'] = config['fsky']
    qp_config['ntubes'] = config['ntubes']
    qp_config['nyears'] = config['nyears']
    qp_config['initial_band'] = []

    for i in range(len(config['frequency'])):
        #print(config['edges'][i][0], config['edges'][i][-1])
        newedges = np.linspace(config['edges'][i][0], config['edges'][i][-1], qp_nsubs[i]+1)
        #print(newedges)
        newfreqs = (newedges[0:-1]+newedges[1:])/2
        newbandwidth = newedges[1:] - newedges[0:-1]
        newdnu_nu = newbandwidth / newfreqs
        newfwhm = config['fwhm'][i] * config['frequency'][i]/newfreqs
        scalefactor_noise = np.sqrt(qp_nsubs[i]) * fct_subopt(config['frequency'][i]) / qp_effective_fraction[i]
        newdepth_p = config['depth_p'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_i = config['depth_i'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_e = config['depth_e'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_b = config['depth_b'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newell_min = np.ones(qp_nsub[i]) * config['ell_min'][i]
        newnside = np.ones(qp_nsub[i]) * config['nside'][i]
        neweffective_fraction = np.ones(qp_nsub[i]) * qp_effective_fraction[i]
        initial_band = np.ones(qp_nsub[i]) * config['frequency'][i]

        for k in range(qp_nsubs[i]):
            if qp_effective_fraction[i] != 0:
                qp_config['frequency'].append(newfreqs[k])
                if i >= 3:
                    qp_config['depth_p'].append(newdepth_p[k])
                    qp_config['depth_i'].append(newdepth_i[k])
                    qp_config['depth_e'].append(newdepth_e[k])
                    qp_config['depth_b'].append(newdepth_b[k])
                else:
                    qp_config['depth_p'].append(s4_config['depth_p'][i])
                    qp_config['depth_i'].append(s4_config['depth_i'][i])
                    qp_config['depth_e'].append(s4_config['depth_e'][i])
                    qp_config['depth_b'].append(s4_config['depth_b'][i])
                qp_config['fwhm'].append(newfwhm[k])
                qp_config['bandwidth'].append(newbandwidth[k])
                qp_config['dnu_nu'].append(newdnu_nu[k])
                qp_config['ell_min'].append(newell_min[k])
                qp_config['nside'].append(newnside[k])

                qp_config['effective_fraction'].append(neweffective_fraction[k])
                qp_config['initial_band'].append(initial_band[k])
        for k in range(qp_nsubs[i]+1):
            if qp_effective_fraction[i] != 0:
                qp_config['edges'].append(newedges[k])

        #qp_config['depth_p'][:3] = s4_config['depth_p'][:3]
        #qp_config['depth_i'][:3] = s4_config['depth_i'][:3]

    fields = ['frequency', 'depth_p', 'depth_i', 'depth_e', 'depth_b', 'fwhm', 'bandwidth',
              'dnu_nu', 'ell_min', 'nside', 'edges', 'effective_fraction', 'initial_band']
    for j in range(len(fields)):
        qp_config[fields[j]] = np.array(qp_config[fields[j]])

    return qp_config


covmap = get_coverage(0.03, nside=256)
thr = 0.1
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0


qp_nsubs = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5])
qp_effective_fraction = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1])
qp_config=qubicify(s4_config, qp_nsubs, qp_effective_fraction)


plt.figure(figsize=(16, 5))
plt.subplot(1,2,1)
plt.errorbar(s4_config['frequency'], s4_config['depth_p'], xerr=s4_config['bandwidth']/2, fmt='ro', label='CMB-S4')
plt.errorbar(qp_config['frequency'], qp_config['depth_p'], xerr=qp_config['bandwidth']/2, fmt='bo', label='BI')
plt.xlabel('Frequency [GHz]')
plt.ylabel(r'Depth_p [$\mu$K.arcmin]')
plt.title('CMB-S4 Configuration')
plt.legend()
plt.subplot(1,2,2)
plt.errorbar(s4_config['frequency'], s4_config['fwhm'], xerr=s4_config['bandwidth']/2, fmt='ro', label='CMB-S4')
plt.errorbar(qp_config['frequency'], qp_config['fwhm'], xerr=qp_config['bandwidth']/2, fmt='bo', label='BI')
plt.xlabel('Frequency [GHz]')
plt.ylabel('FWHM [arcmin]')
plt.title('CMB-S4 Configuration')
plt.legend()
plt.show()

# CMB-S4 case

In [ ]:
reload(qubicplus)

ref_fwhm=0
skyconfig = {'cmb':42, 'dust':'d0', 'synchrotron':'s0'}

# Computation of maps for CMB-S4 -> (9, 3, npix)

## We put maps at same resolution (0.5 degree) and with noise factor of 10

mapS4_noisy, mapS4_noiseless, noiseS4 = qubicplus.BImaps(skyconfig, s4_config).getskymaps(
                                                                    same_resol=ref_fwhm, 
                                                                    verbose=True, 
                                                                    coverage=covmap, 
                                                                    noise=True, 
                                                                    signoise=1.,
                                                                    beta=[1.54, 1.54, 145, 145])

# QUBIC+ case

In [ ]:
mapqp_noisy, mapqp_noiseless, noiseqp = qubicplus.BImaps(skyconfig, qp_config).getskymaps(
                                                                    same_resol=ref_fwhm,
                                                                    verbose=True, 
                                                                    coverage=covmap, 
                                                                    noise=True, 
                                                                    signoise=1.,
                                                                    beta=[1.54, 1.54, 145, 145])

In [ ]:
comp = [fgb.component_model.Dust(nu0=145., units='K_CMB'), 
        fgb.component_model.CMB(), 
        fgb.component_model.Synchrotron(nu0=145)]

thr = 0
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0

# QUBIC+
instr = get_instrument('Qubic+')
instr.fwhm = np.ones(33)*ref_fwhm*60

r_qp=separate(comp, instr, mapqp_noisy[:, :, pixok], tol=1e-6)

# CMB-S4
instr = get_instrument('CMBS4')
instr.fwhm = np.ones(9)*ref_fwhm*60


r_s4=separate(comp, instr, mapS4_noisy[:, :, pixok], tol=1e-6)

print('QUBIC+ -> ', r_qp.x)
print('CMB-S4 -> ', r_s4.x)

# Maps reconstructed

In [ ]:
truemaps=s4bi.get_component_maps_from_parameters({'cmb':42, 'dust':'d0', 'synchrotron':'s0'}, 
                                                 s4_config, 
                                                 nu0=145, 
                                                 fsky=0.03, 
                                                 nside=256, 
                                                 betad0=1.54, 
                                                 betad1=None, 
                                                 nubreak=None, 
                                                 temp=20, 
                                                 betapl=-3)

In [ ]:
for i in range(3):
    truemaps[i][:, :, ~pixok] = hp.UNSEEN

mapcmb_s4_est = np.ones((3, 12*256**2))*hp.UNSEEN
mapdust_s4_est = np.ones((3, 12*256**2))*hp.UNSEEN
mapsync_s4_est = np.ones((3, 12*256**2))*hp.UNSEEN

mapcmb_bi_est = np.ones((3, 12*256**2))*hp.UNSEEN
mapdust_bi_est = np.ones((3, 12*256**2))*hp.UNSEEN
mapsync_bi_est = np.ones((3, 12*256**2))*hp.UNSEEN

mapcmb_s4_est[:, pixok]=r_s4.s[1]
mapdust_s4_est[:, pixok]=r_s4.s[0]
mapsync_s4_est[:, pixok]=r_s4.s[2]

mapcmb_bi_est[:, pixok]=r_qp.s[1]
mapdust_bi_est[:, pixok]=r_qp.s[0]
mapsync_bi_est[:, pixok]=r_qp.s[2]

allcomp_s4 = [mapcmb_s4_est, mapdust_s4_est, mapsync_s4_est]
allcomp_bi = [mapcmb_bi_est, mapdust_bi_est, mapsync_bi_est]

In [ ]:
istk=1
comp=['CMB', 'Dust', 'Synchrotron']

plt.figure(figsize=(12, 12))

k=1
for i in range(3):
    hp.gnomview(truemaps[i][0, istk], rot=center, reso=15, cmap='jet', sub=(3, 3, k), title=comp[i])
    hp.gnomview(allcomp_s4[i][istk], rot=center, reso=15, cmap='jet', sub=(3, 3, k+3), title=comp[i])
    hp.gnomview(allcomp_bi[i][istk], rot=center, reso=15, cmap='jet', sub=(3, 3, k+6), title=comp[i])
    k+=1
    
plt.show()

ti=['Estimated - True => ']
plt.figure(figsize=(12, 8))
k=1
for i in range(3):
    res_s4=allcomp_s4[i][istk] - truemaps[i][0, istk]
    res_s4[~pixok]=hp.UNSEEN
    
    res_bi=allcomp_bi[i][istk] - truemaps[i][0, istk]
    res_bi[~pixok]=hp.UNSEEN
    
    hp.gnomview(res_s4, rot=center, reso=15, cmap='jet', sub=(2, 3, k), title=ti[0] + comp[k-1] + ' - S4')
    hp.gnomview(res_bi, rot=center, reso=15, cmap='jet', sub=(2, 3, k+3), title=ti[0] + comp[k-1] + ' - BI')
    k+=1
plt.show()

In [ ]:
noise_est_s4=mapS4_noisy[5, istk]-allcomp_s4[0][istk]-allcomp_s4[1][istk]-allcomp_s4[2][istk]
noise_est_bi=mapqp_noisy[15, istk]-allcomp_bi[0][istk]-allcomp_bi[1][istk]-allcomp_bi[2][istk]

In [ ]:
plt.figure(figsize=(16, 10))

plt.scatter(0, 0, color='red', s=50, label='S4')
plt.scatter(noise_est_s4[pixok], noiseS4[5, istk, pixok], color='red', s=5, alpha=1)
plt.plot([-1,1], [-1, 1], '-r', label=r'$y=x$')

plt.xlabel('Noise estimated', fontsize=15)
plt.ylabel('True Noise', fontsize=15)
plt.title('S4', fontsize=15)

plt.scatter(0, 0, color='blue', s=50, label='BI')
plt.scatter(noise_est_bi[pixok], noiseqp[15, istk, pixok], color='blue', s=10, alpha=0.05)
plt.plot([-2,2], [-2, 2], '-r')

plt.xlabel('Noise estimated', fontsize=15)
plt.ylabel('True Noise', fontsize=15)
plt.title('BI', fontsize=15)
plt.legend(frameon=False, fontsize=25)

plt.show()

# 2 betas

In [ ]:
reload(qubicplus)

ref_fwhm=0
skyconfig = {'cmb':42, 'dust':'d02b'}
nubreak=260

# Computation of maps for CMB-S4 -> (9, 3, npix)

## We put maps at same resolution (0.5 degree) and with noise factor of 10

mapS4_noisy, mapS4_noiseless, noiseS4 = qubicplus.BImaps(skyconfig, s4_config).getskymaps(
                                                                    same_resol=ref_fwhm, 
                                                                    verbose=True, 
                                                                    coverage=covmap, 
                                                                    noise=True,
                                                                    beta=[1.44, 1.64, nubreak, 145])

In [ ]:
mapqp_noisy, mapqp_noiseless, noiseqp = qubicplus.BImaps(skyconfig, qp_config).getskymaps(
                                                                    same_resol=ref_fwhm, 
                                                                    verbose=True, 
                                                                    coverage=covmap, 
                                                                    noise=True,
                                                                    beta=[1.44, 1.64, nubreak, 145])

In [ ]:
reload(fgb.component_model)

comp = [fgb.component_model.Dust_2b(nu0=145, units='K_CMB'), 
        fgb.component_model.CMB()]#, 
        #fgb.component_model.Synchrotron(nu0=145, beta_pl=-3)]

thr = 0
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0

# QUBIC+
instr = get_instrument('Qubic+')
instr.frequency=qp_config['frequency']
instr.fwhm = np.ones(33)*ref_fwhm*60
instr.depth_i=qp_config['depth_i']
instr.depth_p=qp_config['depth_p']

r_qp=separate(comp, instr, mapqp_noisy[:, :, pixok])

# CMB-S4
instr = get_instrument('CMBS4')
instr.fwhm = np.ones(9)*ref_fwhm*60
r_s4=separate(comp, instr, mapS4_noisy[:, :, pixok])

In [ ]:
print('True   -> ', [1.44, 1.64, nubreak, -3])
print('BI-S4 -> ', r_qp.x)
print('CMB-S4 -> ', r_s4.x)

# Estimation of dust

In [ ]:
reload(s4bi)


all_comp_true=s4bi.get_component_maps_from_parameters({'cmb':42, 'dust':'d02b', 'synchrotron':'s0'}, 
                                                      qp_config, 
                                                      nu0=145, 
                                                      fsky=0.03, 
                                                      nside=256, 
                                                      betad0=1.44, 
                                                      betad1=1.64, 
                                                      nubreak=260, 
                                                      temp=20, 
                                                      betapl=-3)

all_comp_s4=s4bi.get_component_maps_from_parameters({'cmb':42, 'dust':'d02b', 'synchrotron':'s0'}, s4_config, 145, 
                                   fsky=0.03, 
                                   nside=256, 
                                   betad0=r_s4.x[0], 
                                   betad1=r_s4.x[1], 
                                   nubreak=r_s4.x[2], 
                                   temp=20, 
                                   betapl=r_s4.x[-1])

all_comp_bi=s4bi.get_component_maps_from_parameters({'cmb':42, 'dust':'d02b', 'synchrotron':'s0'}, qp_config, 145, 
                                   fsky=0.03, 
                                   nside=256, 
                                   betad0=r_qp.x[0], 
                                   betad1=r_qp.x[1], 
                                   nubreak=r_qp.x[2], 
                                   temp=20, 
                                   betapl=r_qp.x[-1])


In [ ]:
for i in range(3):
    all_comp_true[i][:, :, ~pixok] = hp.UNSEEN

mapcmb_s4_est = np.ones((3, 12*256**2))*hp.UNSEEN
mapdust_s4_est = np.ones((3, 12*256**2))*hp.UNSEEN
#mapsync_s4_est = np.ones((3, 12*256**2))*hp.UNSEEN

mapcmb_bi_est = np.ones((3, 12*256**2))*hp.UNSEEN
mapdust_bi_est = np.ones((3, 12*256**2))*hp.UNSEEN
#mapsync_bi_est = np.ones((3, 12*256**2))*hp.UNSEEN

mapcmb_s4_est[:, pixok]=r_s4.s[1]
mapdust_s4_est[:, pixok]=r_s4.s[0]
#mapsync_s4_est[:, pixok]=r_s4.s[2]

mapcmb_bi_est[:, pixok]=r_qp.s[1]
mapdust_bi_est[:, pixok]=r_qp.s[0]
#mapsync_bi_est[:, pixok]=r_qp.s[2]

all_comp_2beta_s4 = [mapcmb_s4_est, mapdust_s4_est]#, mapsync_s4_est]
all_comp_2beta_bi = [mapcmb_bi_est, mapdust_bi_est]#, mapsync_bi_est]

In [ ]:
istk=1

plt.figure(figsize=(16, 10))
hp.gnomview(all_comp_true[1][0, istk], rot=center, reso=15, cmap='jet', sub=(2, 3, 1), title='From FGB - True')
hp.gnomview(mapdust_s4_est[istk], rot=center, reso=15, cmap='jet', sub=(2, 3, 2), title='From FGB - S4')
hp.gnomview(mapdust_bi_est[istk], rot=center, reso=15, cmap='jet', sub=(2, 3, 3), title='From FGB - BI')
hp.gnomview(all_comp_true[1][0, istk], rot=center, reso=15, cmap='jet', sub=(2, 3, 4), title='Reconstruction with estimated parameters - True')

all_comp_s4[1][0, :, ~pixok]=hp.UNSEEN
all_comp_bi[1][0, :, ~pixok]=hp.UNSEEN

hp.gnomview(all_comp_s4[1][0, istk], rot=center, reso=15, cmap='jet', sub=(2, 3, 5), title='Reconstruction with estimated parameters - S4')
hp.gnomview(all_comp_bi[1][0, istk], rot=center, reso=15, cmap='jet', sub=(2, 3, 6), title='Reconstruction with estimated parameters - BI')
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))

plt.subplot(1, 2, 1)

plt.scatter(0, 0, color='red', s=50, label='S4')
plt.scatter(mapdust_s4_est[istk, pixok], all_comp_true[1][0, istk, pixok], color='red', s=5, alpha=1)
plt.plot([-2,5], [-2,5], '--k', lw=3, label=r'$y=x$')
plt.scatter(0, 0, color='blue', s=50, label='BI')
plt.scatter(mapdust_bi_est[istk, pixok], all_comp_true[1][0, istk, pixok], color='blue', s=10, alpha=1)

plt.xlabel('Dust estimated', fontsize=15)
plt.ylabel('True Dust', fontsize=15)
plt.title('From FGB', fontsize=15)
plt.legend(frameon=False, fontsize=25)

plt.subplot(1, 2, 2)

plt.scatter(0, 0, color='red', s=50, label='S4')
plt.scatter(0, 0, color='blue', s=50, label='BI')
plt.scatter(all_comp_s4[1][0, istk, pixok], all_comp_true[1][0, istk, pixok], color='red', s=5, alpha=1)
plt.scatter(all_comp_bi[1][0, istk, pixok], all_comp_true[1][0, istk, pixok], color='blue', s=10, alpha=1)
plt.plot([-2,5], [-2,5], '--k', lw=3, label=r'$y=x$')

plt.xlabel('Dust estimated', fontsize=15)
plt.ylabel('True Dust', fontsize=15)
plt.title('From reconstruction', fontsize=15)
plt.legend(frameon=False, fontsize=25)
plt.show()

# 1 beta model

In [ ]:
with open('results/beta/params_est_fwhm0.0_500reals.pkl', 'rb') as f:
    data = pickle.load(f)

samples_s4=data['samples_s4']
samples_bi=data['samples_bi']

with open('results/beta/params_est_2beta_fwhm0.0_500reals.pkl', 'rb') as f:
    data = pickle.load(f)

samples_s4_2b=data['samples_s4_2b']
samples_bi_2b=data['samples_bi_2b']

In [ ]:
from getdist import plots, MCSamples

alllabels = [r'\beta_{dust}', 'T', r'\beta_{sync}']
allnames = ['betadust', 'T', 'betasync']

samps = MCSamples(samples=samples_s4.T, names=allnames, labels=alllabels)

sampsBI = MCSamples(samples=samples_bi.T, names=allnames, labels=alllabels)

truths=[1.54, 20, -3]
mkrs = {}
for i in range(len(allnames)):
    mkrs[allnames[i]] = truths[i]

plt.figure()
g = plots.getSubplotPlotter()
g.settings.scaling=False
g.settings.axes_fontsize=12
g.triangle_plot([samps, sampsBI], filled=True, title_limit=None, legend_labels=['S4', 'BI'], markers=mkrs)
plt.show()

# 2 beta model

In [ ]:
#2beta
from getdist import plots, MCSamples

alllabels = [r'\beta_{0}', r'\beta_{1}', r'\nu_{break}', 'T', r'\beta_{sync}']
allnames = ['betadust0', 'betadust1', 'nubreak', 'T', 'betasync']

samps = MCSamples(samples=samples_s4_2b.T, names=allnames, labels=alllabels)

sampsBI = MCSamples(samples=samples_bi_2b.T, names=allnames, labels=alllabels)

truths=[1.44, 1.64, 260, 20, -3]
mkrs = {}
for i in range(len(allnames)):
    mkrs[allnames[i]] = truths[i]

plt.figure(figsize=(16, 16))
g = plots.getSubplotPlotter()
g.settings.scaling=False
g.settings.axes_fontsize=12
g.triangle_plot([samps, sampsBI], filled=True, title_limit=None, legend_labels=['S4', 'BI'], markers=mkrs)
plt.show()

In [ ]:
reload(s4bi)

all_comp=s4bi.get_component_maps_from_parameters({'cmb':42, 'dust':'d0', 'synchrotron':'s0'}, s4_config, 145, 
                                   fsky=0.03, 
                                   nside=256, 
                                   betad0=1.54, 
                                   betad1=1.54, 
                                   nubreak=150, 
                                   temp=20, 
                                   betapl=-3)

In [ ]:
istk=1

plt.figure(figsize=(12, 10))
hp.gnomview(true_dust_maps[0, istk], rot=center, reso=15, sub=(2, 3, 1), cmap='jet', 
            title='True dust map - Q - (1)', unit=r'$\mu K_{CMB}$')
hp.gnomview(new_dust_maps_s4[0, istk], rot=center, reso=15, sub=(2, 3, 2), cmap='jet', 
            title='Reconstructed by CMB-S4 - Q - (2)', min=np.min(true_dust_maps[0, istk, pixok]), 
            max=np.max(true_dust_maps[0, istk, pixok]), unit=r'$\mu K_{CMB}$')
hp.gnomview(new_dust_maps_qp[0, istk], rot=center, reso=15, sub=(2, 3, 3), cmap='jet', 
            title='Reconstructed by BI-S4 - Q - (3)', min=np.min(true_dust_maps[0, istk, pixok]), 
            max=np.max(true_dust_maps[0, istk, pixok]), unit=r'$\mu K_{CMB}$')

res_truetrue=true_dust_maps[0, istk]-true_dust_maps[0, istk]
res_s4true=new_dust_maps_s4[0, istk]-true_dust_maps[0, istk]
res_bitrue=new_dust_maps_qp[0, istk]-true_dust_maps[0, istk]
res_truetrue[~pixok]=hp.UNSEEN
res_s4true[~pixok]=hp.UNSEEN
res_bitrue[~pixok]=hp.UNSEEN

hp.gnomview(res_truetrue, rot=center, reso=15, sub=(2, 3, 4), cmap='jet', title='(1) - (1)', unit=r'$\mu K_{CMB}$')
hp.gnomview(res_s4true, rot=center, reso=15, sub=(2, 3, 5), cmap='jet', 
            min=-np.std(true_dust_maps[0, istk, pixok]), 
            max=np.std(true_dust_maps[0, istk, pixok]), title='(1) - (2)', unit=r'$\mu K_{CMB}$')
hp.gnomview(res_bitrue, rot=center, reso=15, sub=(2, 3, 6), cmap='jet', 
            min=-np.std(true_dust_maps[0, istk, pixok]), 
            max=np.std(true_dust_maps[0, istk, pixok]), title='(1) - (3)', unit=r'$\mu K_{CMB}$')
plt.show()

In [ ]:
qp_config['frequency']

In [ ]:
s4_config['frequency']